In [1]:
import pymssql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from pandas import DataFrame, to_datetime
from datetime import date, timedelta
from prophet import Prophet
import datetime as dt

def DB_connection(tank_seq):
    conn = pymssql.connect(server='sql16ssd-014.localnet.kr', user='i2on11_admin', password='root0826', database='i2on11_admin')
    cursor = conn.cursor()

    conn_sql = 'SELECT signal_time, tank_remain_volume FROM gas_tank_volume_history where tank_seq = ' + tank_seq + 'ORDER BY signal_time DESC'

    # 쿼리 실행
    cursor.execute(conn_sql)

    # 결과 가져오기
    rows = cursor.fetchall()

    # 연결 닫기
    conn.close()
    import datetime

    days_ago = datetime.datetime.now().date() - datetime.timedelta(days=30)
    days_ago_datetime = datetime.datetime.combine(days_ago, datetime.datetime.min.time())

    d_h = pd.DataFrame(rows, columns=['ds', 'y'])
    d_h['ds'] = pd.to_datetime(d_h['ds'])
    d_h.set_index('ds', inplace=True)
    d_h.sort_index(ascending=True, inplace=True)
    d_h = d_h[d_h['y'] > 5]
    d_h = d_h[d_h.index >= days_ago_datetime]

    diff_d_h = d_h.copy()
    diff_d_h['y'] = diff_d_h['y'].diff()
    diff_d_h = diff_d_h.dropna()

    try:
        recent_datetime = diff_d_h[diff_d_h['y'] > 15].index.max()
        d_h = d_h[d_h.index > recent_datetime]
    except:
        pass

    d_h.reset_index(inplace=True)
    diff_d_h.reset_index(inplace=True)
    
    return d_h, diff_d_h

In [2]:
def prophet_prediction(d_h, diff_d_h):
    # Prophet 객체 생성
    model = Prophet()

    # 데이터로 모델 훈련
    model.fit(d_h)

    # 향후 15일 동안의 예측
    future = model.make_future_dataframe(periods=15)
    forecast = model.predict(future)
#     fig1 = model.plot(forecast)
#     fig2 = model.plot_components(forecast)
#     print(forecast[['ds','yhat']])
    forecast['ds'] = pd.to_datetime(forecast['ds'])
    dh_index_max = d_h['ds'].max()
    latest_forecast = forecast[forecast['ds'] >= pd.to_datetime(dh_index_max)]

    prediction = {}
    for _, item in latest_forecast.iterrows():  # Iterate over rows using .iterrows()
        last_set = {item['ds']: item['yhat']}
        prediction.update(last_set)
    return prediction


In [3]:
def prediction_by_tank_seq(tank_seq):
    d_h, diff_d_h = DB_connection(tank_seq)
    pred_h = prophet_prediction(d_h, diff_d_h)
    return pred_h

prediction_by_tank_seq('327')


13:37:25 - cmdstanpy - INFO - Chain [1] start processing
13:37:25 - cmdstanpy - INFO - Chain [1] done processing


{Timestamp('2023-06-12 09:15:49'): 75.00002000254348,
 Timestamp('2023-06-13 09:15:49'): 73.67890121222406,
 Timestamp('2023-06-14 09:15:49'): 72.35778242402792,
 Timestamp('2023-06-15 09:15:49'): 71.03666359579987,
 Timestamp('2023-06-16 09:15:49'): 69.71554480548042,
 Timestamp('2023-06-17 09:15:49'): 68.39442597725241,
 Timestamp('2023-06-18 09:15:49'): 67.07330718905621,
 Timestamp('2023-06-19 09:15:49'): 65.75218836082826,
 Timestamp('2023-06-20 09:15:49'): 64.43106957050867,
 Timestamp('2023-06-21 09:15:49'): 63.1099507422808,
 Timestamp('2023-06-22 09:15:49'): 61.78883191405279,
 Timestamp('2023-06-23 09:15:49'): 60.46771312373335,
 Timestamp('2023-06-24 09:15:49'): 59.14659429550531,
 Timestamp('2023-06-25 09:15:49'): 57.825475507309164,
 Timestamp('2023-06-26 09:15:49'): 56.50435671698969,
 Timestamp('2023-06-27 09:15:49'): 55.18323788876168}

In [ ]:
from flask import Flask
from flask import request
from flask import jsonify
from datetime import datetime
from flask import render_template
import time
from multiprocessing.dummy import Pool as ThreadPool
import concurrent.futures

app = Flask(__name__)
executor = concurrent.futures.ThreadPoolExecutor()

@app.route('/test')
def seaqq():
    data = request.form
    exception = []
    start_time = time.time()

    tank_list = ['374', '338', '759', '347', '323', '437', '358', '396', '547', '503', '353', '512', '370', '327', '371']
    runningvalue = '<div> tank_list_length : ' + str(len(tank_list)) + '</div>'
    for tank_seq in tank_list:
        try:
            prophet = executor.submit(prediction_by_tank_seq(tank_seq))
            runningvalue = runningvalue + '<div>' + str(prophet) + '</div>'
        except:
            exception.append(str(tank_seq))
        
    end_time = time.time()
    exec_time = end_time - start_time
    
    return render_template('index.html', runningvalue=runningvalue, exec_time=exec_time, exception=exception)
#     return jsonify({'result': runningvalue})

@app.route('/test2')
def seaqq2():
    data = request.form
    exception = []
    start_time = time.time()

    tank_list = ['569', '317', '313', '316', '418', '419', '475', '743', '752', '324', '767', '380', '336', '513', '774']
    runningvalue = '<div> tank_list_length : ' + str(len(tank_list)) + '</div>'
    for tank_seq in tank_list:
        try:
            prophet = executor.submit(prediction_by_tank_seq(tank_seq))
            runningvalue = runningvalue + '<div>' + str(prophet) + '</div>'
        except:
            exception.append(str(tank_seq))
        
    end_time = time.time()
    exec_time = end_time - start_time
    
    return render_template('index.html', runningvalue=runningvalue, exec_time=exec_time, exception=exception)
#     return jsonify({'result': runningvalue})

@app.route('/test3')
def seaqq3():
    data = request.form
    exception = []
    start_time = time.time()
#     time.sleep(5)
    tank_list = ['395', '535', '510', '328', '405', '386', '315', '369', '392', '463', '477', '543', '394', '407', '341']
    runningvalue = '<div> tank_list_length : ' + str(len(tank_list)) + '</div>'
    for tank_seq in tank_list:
        try:
            prophet = executor.submit(prediction_by_tank_seq(tank_seq))
            runningvalue = runningvalue + '<div>' + str(prophet) + '</div>'
        except:
            exception.append(str(tank_seq))
        
    end_time = time.time()
    exec_time = end_time - start_time
    
    return render_template('index.html', runningvalue=runningvalue, exec_time=exec_time, exception=exception)
#     return jsonify({'result': runningvalue})
           
if __name__ == "__main__":
    app.run(host="0.0.0.0", port="5000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.21:5000/ (Press CTRL+C to quit)
13:37:38 - cmdstanpy - INFO - Chain [1] start processing
13:37:38 - cmdstanpy - INFO - Chain [1] done processing
13:37:39 - cmdstanpy - INFO - Chain [1] start processing
13:37:39 - cmdstanpy - INFO - Chain [1] done processing
13:37:39 - cmdstanpy - INFO - Chain [1] start processing
13:37:39 - cmdstanpy - INFO - Chain [1] done processing
13:37:40 - cmdstanpy - INFO - Chain [1] start processing
13:37:40 - cmdstanpy - INFO - Chain [1] start processing
13:37:40 - cmdstanpy - INFO - Chain [1] done processing
13:37:40 - cmdstanpy - INFO - Chain [1] done processing
13:37:40 - cmdstanpy - INFO - Chain [1] start processing
13:37:40 - cmdstanpy - INFO - Chain [1] done processing
13:37:40 - cmdstanpy - INFO - Chain [1] start processing
13:37:41 - cmdstanpy - INFO - Chain [1] start processing
13:37:41 - cmdstanpy - INFO - Chain [1] done processing
13:37:41 - cmdstanpy - INFO - Chain [1] start processing
13: